In [39]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [40]:
#import sklearn as sk
import transformers
#import sentence-transformers
from  datasets import Dataset 
import torch 
import pandas as pd
from transformers import Trainer, TrainingArguments

#from tensorflow import keras

In [41]:



df = pd.read_csv("/home/profniggastein/PycharmProjects/ReducedDataset/gcj2009_balanced_exp.csv")

df = df[["flines", "solution"]].dropna()
df = df.rename(columns={"flines": "code", "solution": "label"})
df["label"] = df["label"].astype(int)


# df = df[["code", "Correct/incorrect"]].dropna()
# df = df.rename(columns={"Correct/incorrect": "label"})
# df["label"] = df["label"].astype(int)


dataset = Dataset.from_pandas(df)


In [42]:
from transformers import RobertaTokenizer
from datasets import Value

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

def preprocess(examples):
    return tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = tokenized_dataset["train"].cast_column("label", Value("float32"))
eval_dataset = tokenized_dataset["test"].cast_column("label", Value("float32"))

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Casting the dataset: 100%|██████████| 600/600 [00:00<00:00, 11901.38 examples/s]


In [43]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=1)


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
training_args = TrainingArguments(
    output_dir="./codebert-correctness-gcj(Bin-exp1)",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=2e-5 ,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

In [45]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs > 0.5).astype(int)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

In [46]:




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.326700
20,0.279300
30,0.271900
40,0.246500
50,0.275600
60,0.259900
70,0.262300
80,0.272700
90,0.266000
100,0.270500


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator i

TrainOutput(global_step=3000, training_loss=0.1886054312388102, metrics={'train_runtime': 19995.3323, 'train_samples_per_second': 1.2, 'train_steps_per_second': 0.15, 'total_flos': 3157304315904000.0, 'train_loss': 0.1886054312388102, 'epoch': 10.0})

In [47]:
    # Save the model
model.save_pretrained("./codebert-correctness-gcj(Bin-exp1)")
tokenizer.save_pretrained("./codebert-correctness-gcj(Bin-exp1)")


('./codebert-correctness-gcj(Bin-exp1)/tokenizer_config.json',
 './codebert-correctness-gcj(Bin-exp1)/special_tokens_map.json',
 './codebert-correctness-gcj(Bin-exp1)/vocab.json',
 './codebert-correctness-gcj(Bin-exp1)/merges.txt',
 './codebert-correctness-gcj(Bin-exp1)/added_tokens.json')

In [49]:
trainer.evaluate()

/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.29624149203300476,
 'eval_accuracy': 0.5016666666666667,
 'eval_f1': 0.6636670416197975,
 'eval_runtime': 92.9422,
 'eval_samples_per_second': 6.456,
 'eval_steps_per_second': 0.807,
 'epoch': 10.0}